In [1]:
import os
import io
from tqdm.notebook import tqdm

In [2]:
BEN_PATH = r'D:\Source\pdf\Benign_PDF'
MAL_PATH = r'D:\Source\pdf\Malicious_PDF'

In [81]:
def parse_pdf(file):
    ret = {
        'header':{},
        'body':{'inv_obj_cnt':0},
        'xref':{},
        'trailer':{}
    }
    
    if isinstance(file, io.BufferedReader):
        buffer = file.read()
    elif isinstance(file, bytes):
        buffer = file
    elif os.path.isfile(file):
        with open(file, 'rb') as f:
            buffer = f.read()
    buffer = buffer.replace("endobj".encode(), "endobj\r\n".encode())
    buffer = buffer.replace("\t".encode(), " ".encode())
    buffer = buffer.replace(" obj".encode(), " obj\r\n".encode())
    buffer = buffer.split('\r\n'.encode())
    data = []
    for i in buffer:
        for j in i.split('\r'.encode()):
            for k in j.split('\n'.encode()):
                data.append(k)
    
    head = 0
    try:
        ret['header'] = data[head].decode()
    except UnicodeDecodeError:
        ret['header'] = data[head]
    while  head < len(data) and 'xref'.encode() not in data[head]:
        # try parse obj
        temp = data[head].split(" ".encode())
        if len(temp) == 3 and temp[-1] == 'obj'.encode():
            try:
                obj_num, obj_version = map(int, temp[:-1])
                ret['body'][obj_num] = {'version':obj_version, 'tags':set()}
                head += 1
                if head >= len(data):
                    break
                while head < len(data) and 'endobj'.encode() not in data[head]:
                    if 'stream'.encode() in data[head]:
                        ret['body'][obj_num]['stream'] = b''
                        head += 1
                        if head >= len(data):
                            ret['body'][obj_num]['actual_length'] = len(ret['body'][obj_num]['stream'])
                            break                        
                        while head < len(data) and 'endstream'.encode() not in data[head]:
                            ret['body'][obj_num]['stream'] += data[head]
                            head += 1
                            if head >= len(data):
                                break
                        ret['body'][obj_num]['actual_length'] = len(ret['body'][obj_num]['stream'])
                    if head == len(data):
                        break
                    temp = data[head].split(" ".encode())
                    if temp[0] == '/Length'.encode() and len(temp) == 2:
                        ret['body'][obj_num]['length'] = int(temp[1])
                    for i in temp:
                        for j in i.split('/'.encode()):
                            ret['body'][obj_num]['tags'].add(j.decode().replace("<<", "").replace(">>", "").replace("[", "").replace("]", ""))
                    head += 1
            except ValueError:
                ret['body']['inv_obj_cnt'] += 1
        head += 1
    return ret

In [85]:
import pickle

In [89]:
jobs = []
for path, _, files in os.walk(MAL_PATH):
    for file in files:
        if os.extsep in file:
            name, ext = file.split(os.extsep, maxsplit=1)
        else:
            name, ext = file, None
        if ext == 'pdf' or ext is None:
            jobs.append(path + os.sep + file)
for job in tqdm(jobs):
    result = parse_pdf(job)
    filename = job.split(os.sep)[-1].split(os.extsep)[0]
    with open(r'E:\Source\pdf\mal' + os.sep + filename + os.extsep + 'pickle', 'wb') as f:
        pickle.dump(result, f)

  0%|          | 0/10649 [00:00<?, ?it/s]

In [90]:
jobs = []
for path, _, files in os.walk(BEN_PATH):
    for file in files:
        if os.extsep in file:
            name, ext = file.split(os.extsep, maxsplit=1)
        else:
            name, ext = file, None
        if ext == 'pdf' or ext is None:
            jobs.append(path + os.sep + file)
for job in tqdm(jobs):
    result = parse_pdf(job)
    filename = job.split(os.sep)[-1].split(os.extsep)[0]
    with open(r'E:\Source\pdf\ben' + os.sep + filename + os.extsep + 'pickle', 'wb') as f:
        pickle.dump(result, f)

  0%|          | 0/103567 [00:00<?, ?it/s]

In [74]:
print(result['body'][7])

{'version': 0, 'tags': {'', '0', 'Count', 'Pages', '1', 'Kids28', 'R', 'Type'}}
